# Librerías

In [90]:
import calendar as cd
import pandas as pd
import numpy as np
import math
import psycopg2
import matplotlib.pyplot as plt
from collections import OrderedDict
import json
from datetime import datetime
import query


# Traer datos

In [146]:
with open('credentials_postgre.json') as file:
	credentials = json.load(file)
conn_sql = psycopg2.connect(user = credentials["postgre"]['user'],
                            password = credentials["postgre"]['password'],
                            host = credentials["postgre"]['host'],
                            port = credentials["postgre"]['port'],
                            database = credentials["postgre"]['database'])
query_sshh = query.query_sshh
query_viajes = query.query_viajes

#Traemos las tablas básicas:
#Servicios históricos (SQL)
sshh = pd.read_sql(query_sshh, conn_sql)
#Viajes (SQL)
viajes = pd.read_sql(query_viajes, conn_sql)
#Transición y precio histórico (EXCEL)
transicion = pd.read_excel("TT y PH.xlsx",sheet_name = "Transición")
transicion = transicion.fillna(0)
precio_historico = pd.read_excel("TT y PH.xlsx",sheet_name = "Precio histórico")
#Renombramos algunas columnas de Transición y Precio Histórico para mayor comodidad.
precio_historico = precio_historico.rename(columns={'precio histórico_id':'precio_historico_id','fecha inicio':'fecha_inicio_precio','fecha término':'fecha_termino_precio'})
transicion = transicion.rename(columns={'id_servicio':'servicio_id'})

#Generamos tablas de interés mediante la combinación de ciertas tablas básicas:
#Tabla de homologación
homologacion = sshh.merge(transicion[["nombre_edp","codigo_comercial","servicio_id"]].reset_index(drop=True),'left','servicio_id')
#Tabla de viajes + características comerciales
viajes_tr_ph = viajes.merge(right=transicion, how='left', on='servicio_id',sort='viaje_id').merge(right=precio_historico, how='left', on='precio_historico_id',sort='viaje_id')
viajes_tr_ph = viajes_tr_ph.drop(["fecha_inicio1"],axis=1)
viajes_tr_ph["fecha_inicio"]=pd.to_datetime(
viajes_tr_ph["fecha_inicio"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)#.dt.tz_localize(None)

viajes_tr_ph["fecha_inicio_cobro"]=pd.to_datetime(
viajes_tr_ph["fecha_inicio_cobro"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)#.dt.tz_localize(None)
viajes_tr_ph["vigencia_cobro"]=pd.to_datetime(
viajes_tr_ph["vigencia_cobro"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)
viajes_tr_ph["vigencia_servicio"]=pd.to_datetime(
viajes_tr_ph["vigencia_servicio"],
errors = 'coerce',
format = '%Y-%m-%d %H:%M:%S'
)

# Función generadora de EDP

In [185]:
def dias(frecuencia,inicio,fin):
    a = cd.Calendar()
    weekdays = {'L-D':range(7),'L-S':range(6),'L-V':range(5),'L-J':range(4),'M-S':(1,2,3,4,5),'V':[4],'X':[2]}
    if frecuencia not in weekdays:
        raise ValueError("Ingrese una frecuencia válida")
    if inicio[5:7]==fin[5:7]:
        return sum([1 for i in a.itermonthdays4(int(inicio[:4]),int(inicio[5:7])) if i[1]==int(inicio[5:7]) and i[2] <= int(fin[8:10]) and i[2] >= int(inicio[8:10]) and i[3] in weekdays[frecuencia]])

    elif (int(fin[5:7])-int(inicio[5:7])==1) or (int(inicio[5:7])==12 and fin[5:7] == '01'):
        return sum([1 for i in a.itermonthdays4(int(inicio[:4]),int(inicio[5:7])) if i[1]==int(inicio[5:7]) and i[2] >= int(inicio[8:10])  and i[3] in weekdays[frecuencia]]) + sum([1 for i in a.itermonthdays4(int(inicio[:4]),int(fin[5:7])) if i[1]==int(fin[5:7]) and i[2] <= int(fin[8:10]) and i[3] in weekdays[frecuencia]])
    else:
        raise ValueError("Los meses de inicio y fin deben ser iguales o sucesivos.")

def generar_edp(data,contrato,inicio,fin):
  data_filtrada = data[(data["fecha_inicio"] >= inicio) &
                      (data["fecha_inicio"] <= fin) &
                      (data["nombre"] == contrato) &
                      (data["fecha_inicio"] < data["vigencia_cobro"]) &
                      (data["fecha_inicio"] > data["fecha_inicio_cobro"])].sort_values("fecha_inicio").reset_index(drop=True)

  #Se define este diccionario para luego pasarlo al groupby, lo que permite agrupar según dos valores distintos con una función diferente para cada uno.
  dict_agg={'servicio_id':(lambda x: list(OrderedDict.fromkeys([i for i in x]))),'viaje_id':'count','distancia':sum,'cantidad_vehiculos':max,'km_carga':sum,'km_vacio':sum}
  cantidad = pd.pivot_table(data_filtrada,["servicio_id","viaje_id","distancia","cantidad_vehiculos","km_carga","km_vacio"],index=["nombre_edp","codigo_comercial","Frecuencia","tipo de cobro","fecha_inicio_cobro","vigencia_servicio","km_carga_mensual","km_vacio_mensual","km_mantencion_mensual"],aggfunc=dict_agg).reset_index()
  #return cantidad
  cantidad["distancia"] = cantidad["distancia"].apply(lambda x: round(x,2))
  cantidad = cantidad.reindex(columns = ['nombre_edp','codigo_comercial','Frecuencia','servicio_id','tipo de cobro','cantidad_vehiculos','viaje_id','distancia','fecha_inicio_cobro','vigencia_servicio',"km_carga_mensual","km_vacio_mensual","km_mantencion_mensual","km_carga","km_vacio"])
  cantidad["dias_de_ejecucion"] = [dias(e,inicio,fin) for e in cantidad["Frecuencia"]]
  #e[0]: dias de ejecucion
  #e[1]: fecha inicio cobro
  #e[2]: vigencia servicio
  #e[3]: Frecuencia
  #si fecha_inicio_cobro es menor que inicio_edp y fin_edp es menor que vigencia_servicio, dias_disponibilidad = dias de ejecucion
  #
  cantidad["dias_disponibilidad"] = [e[0] if (e[1] <= datetime.strptime(inicio,'%Y-%m-%d %H:%M:%S') and e[2] >= datetime.strptime(fin,'%Y-%m-%d %H:%M:%S')) else dias(e[3],str(e[1]),fin) if e[1] > datetime.strptime(inicio,'%Y-%m-%d %H:%M:%S') else dias(e[3],inicio,str(e[2])) if e[2] < datetime.strptime(fin,'%Y-%m-%d %H:%M:%S') else 9999 for e in np.array(cantidad[["dias_de_ejecucion","fecha_inicio_cobro","vigencia_servicio","Frecuencia"]])]
  cantidad["disponibilidad"] = cantidad["dias_disponibilidad"]/cantidad["dias_de_ejecucion"]
  cantidad["tramos_a_ejecutar"]=cantidad["servicio_id"].apply(lambda x: len(x)) * cantidad["dias_disponibilidad"]
  cantidad["finalizados"] = cantidad["nombre_edp"].apply(lambda x: sum([1 if (x == fila[0]) and (fila[1]=="FINALIZADO") else 0 for fila in np.array(data_filtrada[["nombre_edp","estado"]])]))
  cantidad["no_iniciados"] = cantidad["nombre_edp"].apply(lambda x: sum([1 if (x == fila[0]) and (fila[1]=="NO_INICIADO") else 0 for fila in np.array(data_filtrada[["nombre_edp","estado"]])]))
  cantidad["anulados"] = cantidad["nombre_edp"].apply(lambda x: sum([1 if (x == fila[0]) and (fila[1]=="ANULADO") else 0 for fila in np.array(data_filtrada[["nombre_edp","estado"]])]))
  '''Fin del reporte operacional. Empiaza creación del comercial.
  '''

  disponibilidad = pd.pivot_table(cantidad,values=["dias_de_ejecucion","dias_disponibilidad"],index=["nombre_edp"],aggfunc=sum).reset_index()
  disponibilidad['disponibilidad'] = round(disponibilidad["dias_disponibilidad"]/disponibilidad["dias_de_ejecucion"],3)
  disponibilidad = disponibilidad.sort_values(by=["nombre_edp"])
  #return disponibilidad
  
  comercial = pd.pivot_table(data_filtrada,index=["nombre_edp","codigo_comercial","tipo de cobro"],values=["distancia","cantidad_vehiculos","viaje_id"],aggfunc={'distancia':sum,'cantidad_vehiculos':max,'viaje_id':'count'}).reset_index()
  #Buscar el precio_historico_id de cada servicio según su código comercial y agregarlos como una columna.
  lista = []
  for i in range(len(comercial)):
    for j in range(len(transicion)):
      if comercial["codigo_comercial"][i] == transicion["codigo_comercial"][j] and datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S') < transicion["vigencia_cobro"][j]:
        lista.append(transicion["precio_historico_id"][j])
        break
  comercial["precio_historico_id"]=pd.Series(lista)
  
  comercial = pd.pivot_table(comercial,values=['viaje_id','distancia'],index=['nombre_edp','tipo de cobro','precio_historico_id','cantidad_vehiculos'],aggfunc=sum).reset_index()
  comercial = comercial.sort_values(by=["nombre_edp"])
  comercial["disponibilidad"] = disponibilidad["disponibilidad"]
  comercial = comercial[["nombre_edp","tipo de cobro","precio_historico_id","cantidad_vehiculos","disponibilidad","viaje_id","distancia"]]
  #return comercial
  columna_1 = []
  columna_2 = []
  columna_3 = []
  columna_4 = []
  columna_5 = []
  columna_6 = []
  columna_7 = []
  columna_8 = []
  columna_9 = []
  columna_10 = []
  columna_11 = []
  columna_12 = []
  for i in range(len(comercial)):
    for j in range(len(precio_historico)):
      if comercial["precio_historico_id"][i] == precio_historico["precio_historico_id"][j]:
        columna_1.append(precio_historico["EDP separados"][j])
        columna_2.append(precio_historico["Vueltas contratadas"][j])
        columna_3.append(precio_historico["costo fijo flota"][j])
        columna_4.append(precio_historico["costo fijo logística"][j])
        columna_5.append(precio_historico["costo unitario vueltas contratadas"][j])
        columna_6.append(precio_historico["costo fijo viaje no realizado"][j])
        columna_7.append(precio_historico["porcentaje costo fijo no realizado"][j])
        columna_8.append(precio_historico["costo variable vuelta flota"][j])
        columna_9.append(precio_historico["costo variable vuelta logística"][j])
        columna_10.append(precio_historico["costo variable vuelta unitario flota"][j])
        columna_11.append(precio_historico["costo variable vuelta unitario logística"][j])
        columna_12.append(precio_historico["costo variable km"][j])
        break
  comercial["EDP separados"]=pd.Series(columna_1)
  comercial["Vueltas contratadas"]=pd.Series(columna_2)
  comercial["costo fijo flota"]=pd.Series(columna_3)
  comercial["costo fijo logística"]=pd.Series(columna_4)
  comercial["costo unitario vueltas contratadas"]=pd.Series(columna_5)
  comercial["costo fijo viaje no realizado"]=pd.Series(columna_6)
  comercial["porcentaje costo fijo no realizado"]=pd.Series(columna_7)
  comercial["costo variable vuelta flota"]=pd.Series(columna_8)
  comercial["costo variable vuelta logística"]=pd.Series(columna_9)
  comercial["costo variable vuelta unitario flota"]=pd.Series(columna_10)
  comercial["costo variable vuelta unitario logística"]=pd.Series(columna_11)
  comercial["costo variable km"]=pd.Series(columna_12)
  comercial["total"]=0
  for i in range(len(comercial)):
    if comercial["tipo de cobro"][i]==1:
      comercial["total"][i] = comercial["costo variable vuelta unitario logística"][i]*comercial["viaje_id"][i]
    elif comercial["tipo de cobro"][i]==4:
      comercial["total"][i] = comercial["costo fijo logística"][i]
    elif comercial["tipo de cobro"][i]==5:
      comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo fijo logística"][i]
    elif comercial["tipo de cobro"][i]==6:
      comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo variable vuelta unitario logística"][i]*comercial["viaje_id"][i]
    elif comercial["tipo de cobro"][i]==8:
      comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo fijo logística"][i] + comercial["viaje_id"][i]*comercial["costo variable vuelta unitario logística"][i]
    elif comercial["tipo de cobro"][i]==9:
      comercial["total"][i] = comercial["costo variable km"][i]*comercial["distancia"][i]
    elif comercial["tipo de cobro"][i]==10:
      comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo fijo logística"][i] + comercial["distancia"][i]*comercial["costo variable km"][i]
    elif comercial["tipo de cobro"][i]==11:
      comercial["total"][i] = comercial["costo unitario vueltas contratadas"][i]*comercial["viaje_id"][i] + (comercial["Vueltas contratadas"][i] - comercial["viaje_id"][i])*comercial["costo fijo viaje no realizado"][i]
    elif comercial["tipo de cobro"][i]==12:
      comercial["total"][i] = comercial["costo variable km"][i]*comercial["distancia"][i]
    #comercial["total"]=cantidad["total"].astype(int)

  #cantidad.
  #np.isnan(cantidad["Vueltas contratadas"])#[1]#.head(10)
  for i in comercial.columns.values:
    if comercial[i].isnull().sum()==len(comercial[i]):
      comercial=comercial.drop([i],axis=1)
  tipos_de_cobro={1:'Cobro por vuelta', 2:'Cobro por vuelta (F+L)', 3:'Cobro por E/S', 4:'Suma alzada',
                  5:'Suma alzada (F+L)', 6:'Suma alzada + cobro por vuelta', 7:'Suma alzada (F+L) + cobro por vuelta', 8:'Suma alzada (F+L) + cobro por tramo',
                  9:'Suma alzada + cobro por km recorrido', 10:'Suma alzada (F+L) + cobro por km', 11:'Suma alzada – porcentaje del costo unitario por vuelta no realizada', 12:'Variable por km'}

  comercial["tipo de cobro"]=comercial["tipo de cobro"].replace(tipos_de_cobro)
  #for i in cantidad.columns.values[7::]:
  #  cantidad[i]=cantidad[i].astype(int)
  comercial["total"]=comercial["total"].astype(int)
  meses = {'01':'enero','02':'febrero','03':'marzo','04':'abril','05':'mayo','06':'junio','07':'julio','08' : 'agosto', '09' : 'septiembre', '10' : 'octubre', '11':'noviembre','12':'diciembre'}
  cantidad["periodo"] = "Del " + inicio[8:10] + " de " + meses[inicio[5:7]] + " al " + fin[8:10] + " de " + meses[fin[5:7]]
  comercial["periodo"] = "Del " + inicio[8:10] + " de " + meses[inicio[5:7]] + " al " + fin[8:10] + " de " + meses[fin[5:7]]
  cantidad=cantidad.drop(columns=["tipo de cobro","fecha_inicio_cobro","vigencia_servicio"])
  cantidad = cantidad[["nombre_edp","codigo_comercial","Frecuencia","servicio_id","cantidad_vehiculos","dias_de_ejecucion","dias_disponibilidad","disponibilidad","tramos_a_ejecutar","viaje_id","finalizados","no_iniciados","anulados","distancia","km_carga_mensual","km_vacio_mensual","km_mantencion_mensual","km_carga","km_vacio","periodo"]]
  

  return [comercial,cantidad,data_filtrada,comercial["nombre_edp"]==disponibilidad["nombre_edp"]]

In [187]:
contrato = "C SCL GOOD YEAR"
inicio = "2021-11-01 00:00:00"
fin =    "2021-11-30 23:59:59"
generar_edp(viajes_tr_ph,contrato,inicio,fin)[0]

C:\Users\TOMAS~1.RET\AppData\Local\Temp/ipykernel_7404/3941792096.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comercial["total"][i] = comercial["costo fijo flota"][i] * comercial["cantidad_vehiculos"][i] * comercial["disponibilidad"][i] + comercial["costo variable vuelta unitario logística"][i]*comercial["viaje_id"][i]


,nombre_edp,tipo de cobro,precio_historico_id,cantidad_vehiculos,disponibilidad,viaje_id,distancia,EDP separados,costo fijo flota,costo variable vuelta logística,costo variable vuelta unitario logística,total,periodo
0,BUS|ALAMEDA |07:00/19:00|LUNES A DOMINGO,Suma alzada + cobro por vuelta,73,1.0,1.0,180,3057.903,No,4733539,29544.698139,14772.349070,7392561,Del 01 de noviembre al 30 de noviembre
1,BUS|EL MONTE|07:00/19:00|LUNES A DOMINGO,Suma alzada + cobro por vuelta,74,1.0,1.0,179,5336.243,No,4733539,45023.895569,22511.947785,8763177,Del 01 de noviembre al 30 de noviembre
2,BUS|WILLIAMS|07:00/19:00|LUNES A DOMINGO,Suma alzada + cobro por vuelta,75,1.0,1.0,180,2643.041,No,4733539,26447.071991,13223.535995,7113775,Del 01 de noviembre al 30 de noviembre
3,MINIBUS|ALAMEDA |07:00/19:00|LUNES A DOMINGO,Suma alzada + cobro por vuelta,76,1.0,0.4,72,278.776,No,4286873,16359.128722,8179.564361,2303677,Del 01 de noviembre al 30 de noviembre
4,TAXIBUS|APOYO PADRE HURTADO|07:00/19:00|LUNES ...,Suma alzada + cobro por vuelta,78,1.0,0.4,72,1009.773,No,4683289,28477.167282,14238.583641,2898493,Del 01 de noviembre al 30 de noviembre
5,TAXIBUS|PADRE HURTADO|07:00/19:00|LUNES A DOMINGO,Suma alzada + cobro por vuelta,77,1.0,1.0,180,-48119.720,No,4683289,28477.167282,14238.583641,7246234,Del 01 de noviembre al 30 de noviembre
6,TAXIBUS|VESPUCIO P14|07:00/19:00|LUNES A DOMINGO,Suma alzada + cobro por vuelta,79,1.0,1.0,180,-46389.447,No,4683289,31101.327862,15550.663931,7482408,Del 01 de noviembre al 30 de noviembre


# Generar EDP

In [196]:
contrato = "C SCL CMPC"
inicio = "2021-11-01 00:00:00"
fin =    "2021-11-30 23:59:59"
dir = "C:/Users/tomas.retamal/Estados de Pago Git/EDP Python/"
writer = pd.ExcelWriter(dir + "EDP_" +contrato[6::]+"_"+fin[:4]+"_"+fin[5:7]+".xlsx")
comercial = generar_edp(viajes_tr_ph,contrato,inicio,fin)[0].rename(columns={'viaje_id':'tramos_ejecutados'})
operacional = generar_edp(viajes_tr_ph,contrato,inicio,fin)[1].rename(columns={'viaje_id':'tramos_ejecutados'})
viajes_edp = generar_edp(viajes_tr_ph,contrato,inicio,fin)[2].rename(columns={'viaje_id':'tramos_ejecutados'})
comercial.to_excel(writer, sheet_name="Comercial",index=False)
operacional.to_excel(writer, sheet_name="Operacional",index=False)
viajes_edp.to_excel(writer,sheet_name="Viajes",index=False)
writer.save()
writer.close()
#SODEXO NESTLÉ FALTAN FRECUENCIAS

C:\Users\TOMAS~1.RET\AppData\Local\Temp/ipykernel_7404/3941792096.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comercial["total"][i] = comercial["costo fijo logística"][i]


In [137]:
viajes_tr_ph[viajes_tr_ph["servicio_id"]==81]

,alias,servicio_id,servicio_historico_id,contador_viajes,viaje_id,fecha_inicio,fecha_termino,region,estado,eliminado,...,IPC logística (SA),diesel logística (SA),IR logística (SA),dólar logística (SA),IPC logística (variable),diesel logística (variable),IR logística (variable),dólar logística (variable),fecha_inicio_precio,fecha_termino_precio
100390,AA CH_S_ADM_CATEMU,81,4501,1459597,3-26-1-141,2022-01-27 17:20:00,2022-01-27 20:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100391,AA CH_S_ADM_CATEMU,81,4501,1472301,3-26-1-142,2022-01-28 17:20:00,2022-01-28 20:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100392,AA CH_S_ADM_CATEMU,81,4501,1448577,3-26-1-140,2022-01-26 17:20:00,2022-01-26 20:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100393,AA CH_S_ADM_CATEMU,81,4501,1443004,3-26-1-139,2022-01-25 17:20:00,2022-01-25 20:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100394,AA CH_S_ADM_CATEMU,81,4501,1432219,3-26-1-138,2022-01-24 17:20:00,2022-01-24 20:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100515,AA CH_S_ADM_CATEMU,81,4501,240123,3-26-1-15,2021-08-04 17:20:00,2021-08-04 21:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100516,AA CH_S_ADM_CATEMU,81,4501,240124,3-26-1-16,2021-08-05 17:20:00,2021-08-05 21:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100517,AA CH_S_ADM_CATEMU,81,4501,240122,3-26-1-14,2021-08-03 17:20:00,2021-08-03 21:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
100518,AA CH_S_ADM_CATEMU,81,4501,240121,3-26-1-13,2021-08-02 17:20:00,2021-08-02 21:45:00,America/Santiago,FINALIZADO,False,...,0.05,NaN,0.95,NaN,0.7,0.3,NaN,NaN,2021-08-01,2022-02-01
